In [1]:
import os, sys
import pandas as pd

In [2]:
import pickle as pk
from lightning import Trainer
from torch.utils.data import DataLoader
import torch 
import numpy as np
import networkx as nx
import copy
from attack_models_sc4 import _edge_group_clustering, GATEncoder, Decoder, GAE, threshold_by_otsu, contrastive_normalization_np_withpower, build_sparse_edge_otsu, build_sparse_edge
from torch_geometric.data import Data
import torch.nn.functional as F
from tia_util import remove_diag_reshape, reconstruct_with_diagonal, contrastive_normalization_np, reconstruct
from sklearn.metrics import f1_score

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
cur_dir = os.getcwd()
metrics_root = f"{cur_dir}\\saved_metrics\\"
topology_path = f"{cur_dir}/topologies/"

In [23]:
TOPOLOGY = ["star", "ring","ER_0.3", "ER_0.5", "ER_0.7",'Abilene', 'brain', 'GÉANT', 'synth50', 'rf1755', 'rf3967', 'atlanta', 'brain',  'cost266', 'dfn-bwin', 'dfn-gwin', 'di-yuan', 'france',  'germany50', 'giul39', 'india35', 'janos-us', 'janos-us-ca', 'newyork', 'nobel-eu', 'nobel-germany', 'nobel-us', 'norway', 'pdh', 'pioro40', 'polska', 'sun', 'ta1', 'ta2', 'zib54']
NUM_CLIENTS = [10, 20,30,12,22,50,79,87, 15, 161, 37, 11, 25, 35, 26, 39, 16, 28, 17, 14,  40,  27, 24, 65, 54]
TOPOLOGY = ["star", "ring","ER_0.3", "ER_0.5", "ER_0.7"]
# NUM_CLIENTS = [10, 20,30]
# DATASET = ["Cifar10no", "Cifar10", "Mnist","FMnist", "imagenet10", "pcam", "svhn"]
DATASET = ["Cifar10"]
MODEL = ["mlp", "mobile","resnet","pf"]
# MODEL = ["mlp", "mobile", "resnet"]
IID = [1]
MAX_EPOCHS = [3]
ALPHA = 0.1
SEED = [42]
all_metrics_sc = os.listdir(metrics_root)

In [24]:
all_metrics_sc = os.listdir(metrics_root)

In [27]:
all_res = []
for dataset in DATASET:
        for model_name in MODEL:
            # dataset and model setting
            for topo in TOPOLOGY:
               for iid in IID:
                    for seed in SEED:
                        for max_epoch in MAX_EPOCHS:
                            for num in NUM_CLIENTS:
                                ROUND =  num
                                toponame_file = f"{topology_path}/{num}_{topo}.pk"
                                if os.path.exists(toponame_file):  
                                    with open(toponame_file, "rb") as f:
                                        G = pk.load(f)
                                        density = nx.density(G)
                                        label = nx.adjacency_matrix(G).todense()
                                else:
                                    continue
                                
                                scenario_name = dataset + "_" + model_name + "_" + topo + "_" + str(iid) + "_" + str(ALPHA) + "_" + str(seed) + "_" + str(max_epoch) + "_" + str(num)
                                if scenario_name in all_metrics_sc:
                                    all_metrics = {}
                                    # for approach in ['_cosine_metric', '_curvature_divergence', '_entropy_metric', '_euclidean_metric', '_jacobian_metric', '_loss_metric']:
                                    try:
                                        for approach in ['_cosine_metric', '_loss_metric']:                                                                          
                                            metric_path = os.path.join(metrics_root,scenario_name, f'{approach}.csv')                                        
                                            metrics = pd.read_csv(metric_path, index_col=0)
                                            all_metrics[approach] = metrics
                                    except:
                                        continue
                                            
                                    value_ls1 = copy.deepcopy(all_metrics['_cosine_metric'])
                                    reshaped1 = contrastive_normalization_np_withpower(value_ls1, eta=3)
                                    value_ls2 = copy.deepcopy(all_metrics['_loss_metric'])
                                    reshaped2 = contrastive_normalization_np_withpower(value_ls2, eta=3)

                                    reshaped = (reshaped1 + (1-reshaped2))/2
                                    # reshaped = reshaped1
                                    
                                    # value_ls = copy.deepcopy(all_metrics[approach])
                                    # reshaped = contrastive_normalization_np_withpower(value_ls, eta=3)
                                    np.fill_diagonal(reshaped, 0)
                                    
                                    # for clustering_name in ["Kmeans", "GMM", "Spectral"]:             
                                    #     infer_con,_ = _edge_group_clustering(reshaped, clustering_name, flag=True, multi_dimensional=False)
                                    #     infer_adj = reconstruct(infer_con)
                                    #     f1 = f1_score(label.flatten(), infer_adj.flatten(), average='weighted')
                                    #     res = {}
                                    #     res['value'] = f1
                                    #     res['alg'] = clustering_name
                                    #     res['dataset'] = dataset
                                    #     res['topo'] = topo                                        
                                    #     res['iid'] = iid
                                    #     res['max_epoch'] = max_epoch
                                    #     res['num'] = num
                                    #     res['density'] = density
                                    #     res['approach'] = approach
                                    #     all_res.append(res)
                                        
                                    sim_matrix_np = np.copy(reshaped)
                                    sim_matrix_np = contrastive_normalization_np_withpower(sim_matrix_np, eta=4)
                                    np.fill_diagonal(sim_matrix_np, 1)
                                    sim_matrix = torch.tensor(sim_matrix_np, dtype=torch.float32)
                                    x = sim_matrix
                                    edge_index, edge_weight = build_sparse_edge(sim_matrix)
                                    # edge_index, edge_weight, sparse_th = build_sparse_edge(sim_matrix)
                                    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight)
                                    encoder = GATEncoder(in_channels=x.size(1), hidden_channels=32, out_channels=16)
                                    decoder = Decoder(input_dim=16)
                                    model = GAE(encoder, decoder)
                                    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
                                    for epoch in range(200):
                                        model.train()
                                        optimizer.zero_grad()
                                        adj_pred = model(data.x, data.edge_index)
                                        loss = F.mse_loss(adj_pred, sim_matrix)
                                        loss.backward()
                                        optimizer.step()
                                        
                                    model.eval()
                                    with torch.no_grad():
                                        adj_pred = model(data.x, data.edge_index)
                                        # edge_matrix, threshold = threshold_by_knee(adj_pred)
                                        # adj_pred = soft_threshold(adj_pred, tau=0.7)
                                        edge_matrix, threshold = threshold_by_otsu(adj_pred)
                                    edge_matrix.fill_diagonal_(0)
                                    f1 = f1_score(label.flatten(), edge_matrix.flatten(), average='weighted')
                                    res = {}
                                    res['value'] = f1
                                    res['alg'] = 'GNN'
                                    res['dataset'] = dataset
                                    res['topo'] = topo                                        
                                    res['iid'] = iid
                                    res['max_epoch'] = max_epoch
                                    res['num'] = num
                                    res['density'] = density
                                    res['approach'] = approach
                                    all_res.append(res)
                                    print(res)

                                    

{'value': 0.7098666666666666, 'alg': 'GNN', 'dataset': 'Cifar10', 'topo': 'star', 'iid': 1, 'max_epoch': 3, 'num': 10, 'density': 0.2, 'approach': '_loss_metric'}
{'value': 0.7721385781911104, 'alg': 'GNN', 'dataset': 'Cifar10', 'topo': 'star', 'iid': 1, 'max_epoch': 3, 'num': 20, 'density': 0.1, 'approach': '_loss_metric'}
{'value': 0.8302356409584639, 'alg': 'GNN', 'dataset': 'Cifar10', 'topo': 'star', 'iid': 1, 'max_epoch': 3, 'num': 30, 'density': 0.06666666666666667, 'approach': '_loss_metric'}
{'value': 0.9900905046786318, 'alg': 'GNN', 'dataset': 'Cifar10', 'topo': 'ring', 'iid': 1, 'max_epoch': 3, 'num': 10, 'density': 0.2222222222222222, 'approach': '_loss_metric'}
{'value': 0.9950540118214551, 'alg': 'GNN', 'dataset': 'Cifar10', 'topo': 'ring', 'iid': 1, 'max_epoch': 3, 'num': 20, 'density': 0.10526315789473684, 'approach': '_loss_metric'}
{'value': 0.9903219072272557, 'alg': 'GNN', 'dataset': 'Cifar10', 'topo': 'ring', 'iid': 1, 'max_epoch': 3, 'num': 30, 'density': 0.068965

In [28]:
res_df = pd.DataFrame(all_res)
res_df.to_csv('sc3_cfaug.csv')